In [2]:
import random
from datetime import datetime, timedelta
from shapely.geometry import Point, Polygon

# Simulated biologging data structure
class BiologgingData:
    def __init__(self, heart_rate, temperature, activity):
        self.heart_rate = heart_rate
        self.temperature = temperature
        self.activity = activity

# Thresholds
HEART_RATE_MIN = 30
HEART_RATE_MAX = 100
TEMPERATURE_MAX = 28.0
ACTIVITY_MIN = 1

# Shipping lane polygon example
shipping_lane = Polygon([
    (10, 10), (20, 10), (20, 20), (10, 20)
])

# Hydrophone stations coordinates
hydrophones = {
    'H1': (5, 5),
    'H2': (15, 5),
    'H3': (25, 25),
    'H4': (30, 10)
}

SOUND_SPEED = 1500  # m/s approx in water

def distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2) ** 0.5

# Simulate biologging data (some random abnormal)
def simulate_biologging_data(abnormal=False):
    if abnormal:
        param = random.choice(['heart_rate', 'temperature', 'activity'])
        if param == 'heart_rate':
            heart_rate = random.choice([random.randint(10, HEART_RATE_MIN - 1), random.randint(HEART_RATE_MAX + 1, 150)])
            temperature = random.uniform(20, 27)
            activity = random.randint(ACTIVITY_MIN, 10)
        elif param == 'temperature':
            heart_rate = random.randint(40, 80)
            temperature = random.uniform(TEMPERATURE_MAX + 1, 35)
            activity = random.randint(ACTIVITY_MIN, 10)
        else:
            heart_rate = random.randint(40, 80)
            temperature = random.uniform(20, 27)
            activity = 0
    else:
        heart_rate = random.randint(HEART_RATE_MIN, HEART_RATE_MAX)
        temperature = random.uniform(20, TEMPERATURE_MAX)
        activity = random.randint(ACTIVITY_MIN, 10)
    return BiologgingData(heart_rate, temperature, activity)

def check_biologging_alert(data):
    alerts = []
    if data.heart_rate < HEART_RATE_MIN or data.heart_rate > HEART_RATE_MAX:
        alerts.append('Heart rate abnormal')
    if data.temperature > TEMPERATURE_MAX:
        alerts.append('Temperature too high')
    if data.activity < ACTIVITY_MIN:
        alerts.append('Activity too low')
    return alerts

def simulate_gps_location(in_danger_zone=False, underwater=False):
    if underwater:
        return None
    if in_danger_zone:
        lat = random.uniform(11, 19)
        lon = random.uniform(11, 19)
    else:
        lat = random.uniform(0, 9)
        lon = random.uniform(0, 9)
    return (lat, lon)

def check_geofence_alert(location):
    if location is None:
        return False
    point = Point(location)
    return shipping_lane.contains(point)

def simulate_acoustic_arrivals(fish_position):
    base_time = datetime.now()
    arrivals = {}
    for hid, loc in hydrophones.items():
        dist = distance(fish_position, loc)
        time_delay = dist / SOUND_SPEED
        noise = random.uniform(-0.001, 0.001)
        arrival_time = base_time + timedelta(seconds=time_delay + noise)
        arrivals[hid] = arrival_time
    return arrivals

def compute_toad(arrivals):
    earliest_hid = min(arrivals, key=arrivals.get)
    earliest_time = arrivals[earliest_hid]
    toads = {hid: (arrivals[hid] - earliest_time).total_seconds() for hid in arrivals}
    return earliest_hid, toads

def simulate_monitoring_cycle(fish_id, biologging_abnormal=False, in_danger_zone=False, underwater=False):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    bio_data = simulate_biologging_data(abnormal=biologging_abnormal)
    bio_alerts = check_biologging_alert(bio_data)
    gps_location = simulate_gps_location(in_danger_zone=in_danger_zone, underwater=underwater)
    geofence_alert = check_geofence_alert(gps_location)
    
    alerts_occurred = False
    
    print(f'Fish ID: {fish_id} | Timestamp: {timestamp}')
    print(f'Biologging data: HR={bio_data.heart_rate}, Temp={bio_data.temperature:.1f}C, Activity={bio_data.activity}')
    
    if gps_location is None:
        # Underwater: acoustic multilateration approximation
        fish_true_pos = (random.uniform(15, 28), random.uniform(15, 28))
        arrivals = simulate_acoustic_arrivals(fish_true_pos)
        earliest_hid, toads = compute_toad(arrivals)
        
        print('GPS Location: No GPS data (fish underwater)')
        print('Acoustic signal arrival timestamps at hydrophones:')
        for hid, t in arrivals.items():
            print(f'  - {hid}: {t.strftime("%H:%M:%S.%f")}')
        print(f'Earliest signal at hydrophone: {earliest_hid}')
        print('Time of Arrival Differences (ToAD) relative to earliest hydrophone (seconds):')
        for hid, diff in toads.items():
            print(f'  - {hid}: {diff:.6f}')
        print(f'Estimated Fish Location (from multilateration/TOAD): approx ({fish_true_pos[0]:.2f}, {fish_true_pos[1]:.2f})')
        print('Status: Biologging sensors active, position estimated via hydrophone network and TOAD data')
    else:
        print(f'GPS Location: {gps_location[0]:.4f}, {gps_location[1]:.4f}')
        
    if bio_alerts:
        alerts_occurred = True
        print('*** HEALTH ALERT ***')
        for alert in bio_alerts:
            print(' -', alert)
        print('Notification sent to Marine Biologist for immediate investigation.')
    
    if geofence_alert:
        alerts_occurred = True
        print('*** COLLISION RISK ALERT ***')
        print('Fish is inside the shipping lane! Notify ship captain.')
        
    if not alerts_occurred:
        print('No alerts triggered.')
    print('-' * 50)

# Example runs for five fish scenarios

print("=== Fish at surface, normal biologging, outside danger zone ===")
simulate_monitoring_cycle(fish_id='F001', biologging_abnormal=False, in_danger_zone=False, underwater=False)
print()
print("=== Fish underwater, normal biologging ===")
simulate_monitoring_cycle(fish_id='F002', biologging_abnormal=False, in_danger_zone=False, underwater=True)
print()
print("=== Fish underwater, abnormal biologging ===")
simulate_monitoring_cycle(fish_id='F003', biologging_abnormal=True, in_danger_zone=False, underwater=True)
print()
print("=== Fish at surface, abnormal biologging, outside danger zone ===")
simulate_monitoring_cycle(fish_id='F004', biologging_abnormal=True, in_danger_zone=False, underwater=False)
print()
print("=== Fish at surface, normal biologging, inside danger zone ===")
simulate_monitoring_cycle(fish_id='F005', biologging_abnormal=False, in_danger_zone=True, underwater=False)

=== Fish at surface, normal biologging, outside danger zone ===
Fish ID: F001 | Timestamp: 2025-07-26 14:55:39
Biologging data: HR=49, Temp=21.1C, Activity=8
GPS Location: 6.8840, 5.8425
No alerts triggered.
--------------------------------------------------

=== Fish underwater, normal biologging ===
Fish ID: F002 | Timestamp: 2025-07-26 14:55:39
Biologging data: HR=94, Temp=26.9C, Activity=1
GPS Location: No GPS data (fish underwater)
Acoustic signal arrival timestamps at hydrophones:
  - H1: 14:55:39.462976
  - H2: 14:55:39.458679
  - H3: 14:55:39.455246
  - H4: 14:55:39.458990
Earliest signal at hydrophone: H3
Time of Arrival Differences (ToAD) relative to earliest hydrophone (seconds):
  - H1: 0.007730
  - H2: 0.003433
  - H3: 0.000000
  - H4: 0.003744
Estimated Fish Location (from multilateration/TOAD): approx (18.47, 18.68)
Status: Biologging sensors active, position estimated via hydrophone network and TOAD data
No alerts triggered.
---------------------------------------------